This is the ensemble of models we took from multiple runs, managed by Optuna. 
We first read the optuna SQLite file, for the run information of several (5,000) models. 

In [28]:
import optuna

study_name: str = "TRIALS_5"
#study_name: str = "SIN_IMC_CircCint"

study = optuna.create_study(
    study_name=study_name,
    storage="sqlite:///homa_studies.db",
    directions=["maximize", "maximize"],
    sampler=optuna.samplers.TPESampler(),
    load_if_exists=True,
)

[I 2023-04-11 20:21:49,017] Using an existing study with name 'TRIALS_5' instead of creating a new one.


In [29]:
import pandas as pd

df = pd.DataFrame()

# for n, trial in enumerate(study.best_trials):
for n, trial in enumerate(study.trials):
    srs = pd.concat([
        pd.Series(
            trial.user_attrs["shap_abnormal"], 
        ),
        pd.Series(
            trial.user_attrs["shap_healty"], 
        ),
        pd.Series(trial.values[0], index=["AUC"]) # ,"PheScore"])
        ], 
        keys = ["SHAP_Abnormal","SHAP_Healty", "StudyValues"]
    )
    df[f"Trial_{n}"] = srs 


/tmp/ipykernel_619059/2678715819.py:18: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/tmp/ipykernel_619059/2678715819.py:18: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/tmp/ipykernel_619059/2678715819.py:18: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/tmp/ipykernel_619059/2678715819.py:18: Per

In [30]:
lst_seeds = [ trial.params["seed"] for trial in study.trials ]
lst_values = [ {"AUC" : trial.values[0] , "Phe" : trial.values[1]} for trial in study.trials ]

df3 = pd.DataFrame(lst_values)
df3["seeds"] = pd.Series(lst_seeds)

df = df.T # Ease of use

lim_AUC : float = 0.90 # THIS IS THE LIMIT WE USE
df2 = df.where( df["StudyValues"]["AUC"] > lim_AUC ).dropna()

We package this info into DataFrames, for ease of use. 

In [31]:
# This exists for the plots below
names_rename = {
    'gender' : 'Sex',
    'BMI' : 'BMI',
    'Waist circumference' : 'Waist Circumference',
    'ATPII/AHA/IDF' : 'ATPII/AHA/IDF',
    'Phenylalax' : 'PheC',
    'Glupromx' : 'Glucose',
    'Insuprom' : 'Insulin',
    'Tyrosinax' : 'Tyrosine',
    'Alanine' : 'Alanine',
    'Aspartate' : 'Aspartate',
    'Glutamate' : 'Glutamate',
    'Leucine' : 'Leucine',
    'Ornithine' : 'Ornithine',
    'Proline' : 'Proline',
    'Tyrosine' : 'Tyrosine',
    'Free Carnitine' : 'Free Carnitine',
    'Propionylcarnitine' : 'Propionylcarnitine',
    'Isovalerylcarnitine' : 'Isovalerylcarnitine',
    'Tiglilcarnitine' : 'Tiglilcarnitine',
    'Me-Glutarylcarnitine' : 'Me-Glutarylcarnitine',
    'Decanoylcarnitine' : 'Decanoylcarnitine',
    'Tetradecanoylcarnitine' : 'Tetradecanoylcarnitine',
    '3-OH-Isovalerylcarnitine' : '3-OH-Isovalerylcarnitine',
    '3-OH-Palmitoylcarnitine' : '3-OH-Palmitoylcarnitine',
    'Linoleoilcarnitine' : 'Linoleoilcarnitine',
    'Arginine' : 'Arginine',
    'Citrulline' : 'Citrulline',
    'Glycine' : 'Glycine',
    'Methionine' : 'Methionine',
    'Phenylalanine' : 'Phenylalanine',
    'Succinylacetone' : 'Succinylacetone',
    'Valine' : 'Valine',
    'Acetylcarnitine' : 'Acetylcarnitine',
    'Butyrylcarnitine' : 'Butyrylcarnitine',
    'Glutarylcarnitine' : 'Glutarylcarnitine',
    'Hexanoylcarnitine' : 'Hexanoylcarnitine',
    'Octanoylcarnitine' : 'Octanoylcarnitine',
    'Dodecanoylcarnitine' : 'Dodecanoylcarnitine',
    'Tetradecenoylcarnitine' : 'Tetradecenoylcarnitine',
    'Palmitoylcarnitine' : 'Palmitoylcarnitine',
    'Stearoylcarnitine' : 'Stearoylcarnitine',
    '3-OH-Linoleoylcarnitine' : '3-OH-Linoleoylcarnitine',
    'PROTEIN_Avg_(G)' : 'Protein consumption',
    'Protein_natural' : 'Natural protein',
    'SP_Protein' : 'Supplemented protein',
    'FAT_avg(G)' : 'Fat consumption average',
}

In [32]:
fig = optuna.visualization.plot_pareto_front(study, target_names=["AUC","Phenylalanine Contribution"])

fig.add_vrect(x0=lim_AUC, x1=1.0, line_width=1, fillcolor="teal", opacity=0.2)

fig.update_layout(
    title_text="Pareto-front Plot", 
    yaxis_tickformat=".2f",
    template='plotly_white',
)

DPI = 320

fig.write_image("fig/pareto_plots.svg", width=3*DPI, height=2*DPI)

fig.show()

So, we take these models with AUC > 0.9. 

In [33]:
import plotly.express as px

# Just because we liked this palette
custom_colorscale = ["rgb(30, 144, 255)", "rgb(148, 0, 211)", "rgb(255, 20, 147)"]

SORTED_MEANS = df2["SHAP_Abnormal"].drop(["ATPII/AHA/IDF"], axis='columns').mean().sort_values(ascending=False)

relabels = { idx : f"{names_rename[idx]}"  for idx in SORTED_MEANS.index }

fig_abnormal = px.parallel_coordinates(df2["SHAP_Abnormal"],
    dimensions=df2["SHAP_Abnormal"].drop(["ATPII/AHA/IDF"], axis='columns').mean().sort_values(ascending=False).index[:13], 
    color="Phenylalax", 
    labels= relabels,
    color_continuous_scale=custom_colorscale,
    title = f"Model contributions -  'HOMA Abnormal' - {lim_AUC} < AUC ({df2.shape[0]} models)"
    )

fig_abnormal.update_layout(
    yaxis_tickformat=".2f",
    coloraxis_colorbar=dict(title="Phe. weight"),
)

# Save as SVG, we can save as PNG later
fig_abnormal.write_image("fig/abnormal_ranks.svg", width=4.5*DPI, height=1.5*DPI, scale = 1.0)

fig_abnormal.show()

The base observation is that the IMC variable takes a high weight in most models, and there is a sort of bi-modal weight to the PheC.
Some models do not even utilise it, while others count for ~20% of the final weight. 

In [35]:
fig_healty = px.parallel_coordinates(df2["SHAP_Healty"],
    dimensions=df2["SHAP_Healty"].drop(["ATPII/AHA/IDF"], axis='columns').mean().sort_values(ascending=False).index[:13], 
    color="Phenylalax", 
    labels= relabels,
    color_continuous_scale=custom_colorscale,
    title = f"Model contributions -  'HOMA Healty' - {lim_AUC} < AUC ({df2.shape[0]} models)"
    )

fig_healty.write_image("fig/healty_ranks.svg", width=4.5*DPI, height=1.5*DPI, scale = 1.0)

fig_healty.show()

For "healty" samples, the first variables remain the same in the first places, althought the weight assigned to BMI varies greatly. 